# Finding a Metric to Describe Asymmetry in Steering Drug Applications

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pprint import pprint as pp

import os
import sys

import xlwt

from ggplot import *

Initialize fitness values for 15 drugs...

In [ ]:
AMPf = np.array([1.851, 2.082, 1.948, 2.434, 2.024, 2.198, 2.033, 0.034, 1.57, 2.165, 0.051, 0.083, 2.186, 2.322, 0.088, 2.821])
AMf  = np.array([1.778, 1.782, 2.042, 1.752, 1.448, 1.544, 1.184, 0.063, 1.72, 2.008, 1.799, 2.005, 1.557, 2.247, 1.768, 2.047])
CECf = np.array([2.258, 1.996, 2.151, 2.648, 2.396, 1.846, 2.23, 0.214, 0.234, 0.172, 2.242, 0.093, 2.15, 0.095, 2.64, 0.516])
CTXf = np.array([0.16, 0.085, 1.936, 2.348, 1.653, 0.138, 2.295, 2.269, 0.185, 0.14, 1.969, 0.203, 0.225, 0.092, 0.119, 2.412])
ZOXf = np.array([0.993, 0.805, 2.069, 2.683, 1.698, 2.01, 2.138, 2.688, 1.106, 1.171, 1.894, 0.681, 1.116, 1.105, 1.103, 2.591])
CXMf = np.array([1.748, 1.7, 2.07, 1.938, 2.94, 2.173, 2.918, 3.272, 0.423, 1.578, 1.911, 2.754, 2.024, 1.678, 1.591, 2.923])
CROf = np.array([1.092, 0.287, 2.554, 3.042, 2.88, 0.656, 2.732, 0.436, 0.83, 0.54, 3.173, 1.153, 1.407, 0.751, 2.74, 3.227])
AMCf = np.array([1.435, 1.573, 1.061, 1.457, 1.672, 1.625, 0.073, 0.068, 1.417, 1.351, 1.538, 1.59, 1.377, 1.914, 1.307, 1.728])
CAZf = np.array([2.134, 2.656, 2.618, 2.688, 2.042, 2.756, 2.924, 0.251, 0.288, 0.576, 1.604, 1.378, 2.63, 2.677, 2.893, 2.563])
CTTf = np.array([2.125, 1.922, 2.804, 0.588, 3.291, 2.888, 3.082, 3.508, 3.238, 2.966, 2.883, 0.89, 0.546, 3.181, 3.193, 2.543])
SAMf = np.array([1.879, 2.533, 0.133, 0.094, 2.456, 2.437, 0.083, 0.094, 2.198, 2.57, 2.308, 2.886, 2.504, 3.002, 2.528, 3.453])
CPRf = np.array([1.743, 1.662, 1.763, 1.785, 2.018, 2.05, 2.042, 0.218, 1.553, 0.256, 0.165, 0.221, 0.223, 0.239, 1.811, 0.288])
CPDf = np.array([0.595, 0.245, 2.604, 3.043, 1.761, 1.471, 2.91, 3.096, 0.432, 0.388, 2.651, 1.103, 0.638, 0.986, 0.963, 3.268])
TZPf = np.array([2.679, 2.906, 2.427, 0.141, 3.038, 3.309, 2.528, 0.143, 2.709, 2.5, 0.172, 0.093, 2.453, 2.739, 0.609, 0.171])
FEPf = np.array([2.59, 2.572, 2.393, 2.832, 2.44, 2.808, 2.652, 0.611, 2.067, 2.446, 2.957, 2.633, 2.735, 2.863, 2.796, 3.203])

Initialize code:

"States" are the genotype that a drug can be in ranging from '0000' to '1111' and is used to index the fitness values.

The Hamming Distance is a computer science term used to check the total differences between two strings --- or rather minimum number of substituions between them. For example, '0010' $\rightarrow$ '0011' has a Hamming Distance of 1, while '1101' $\rightarrow$ '0000' has a Hamming Distance of 3.

This is used in calculating the transition probability matrix described as follows:

$$
P(i\rightarrow j) = \left\{
        \begin{array}{ll}
            \frac{(f(j)-f(i)^R}{\sum (f(g)-f(i))^R} & \quad f(j)>f(i) \mathrm{\:and\:HammingDistance}(i,j) = 1 \\
            0 & \quad \mathrm{otherwise}
        \end{array}
    \right.
$$

where $g\in \{0,1\}^N$

The Drug class has parameters:
- name = the name of the drug
- Fit = the array of fitness values
- states = the states corresponding to those fitness values
- LFP = the state that is the lowest fitness peak
- R = biasing factor (default set to 0)

The Drug class can call methods to return all the above variables in addition to:
- tMat = the transition matrix calculated from the above formula
- LPFf = the fitness value corresponding to the lowest fitness peak

In [ ]:
# Assign State Binary Genotypes for labels
N = 4  # Bit Count of States
states = []
for i in range(16):
    b = format(i, 'b')
    bB = b.zfill(N)
    states.append(bB)

# Define the Hamming Distance
def hamDist(Si, Sj):
    return sum(ei != ej for ei, ej in zip(Si, Sj))

# Zero State of equal probabilities
S0 = np.array(np.ones(len(states))/2**N)

# Make a drug class
class Drug:
    # This will make it easy to call certain qualities
    # Arguments:
    # name is a string
    # Fit is an array of the fitness values
    # States is the binary genotype that corresponds to the bins
    # R is the biasing variable that we set to 0
    def __init__(self, name, Fit, states, LFP, R=0):
        self.name = name
        self.Fit = np.array(Fit)
        self.states = states
        self.LFP = LFP
        self.R = R
        
        # The following code is to calculate the transition matrix
        P = np.zeros([len(self.states),len(self.states)])
    
        for i in range(len(self.states)):
            for j in range(len(self.states)):
                if self.Fit[j] >= self.Fit[i] and hamDist(self.states[i], self.states[j]) == 1:
                    P[i, j] = (self.Fit[j] - self.Fit[i])**R
                    d = 0
                    for s in range(len(self.states)):
                        if hamDist(self.states[i], self.states[s]) == 1 and self.Fit[s] > self.Fit[i]:
                            d = d + (self.Fit[s] - self.Fit[i])**R
                    P[i, j] = P[i, j]/d
                    
                    
        # if there is no probability to reach any other state,
        # there is 100% probability to remain in the initial state.
        for p in range(len(P)):
            spots = np.where(P[p] != 0)
            if len(spots[0])==0:
                P[p, p]=1

        self.tMat = np.matrix(P)
        self.LPFf = self.Fit[int(self.LFP, 2)]

In [ ]:
AMP = Drug('Ampicillin', AMPf, states, '0110')
AM = Drug('Amoxicillin', AMf, states, '0010')
CEC = Drug('Cefaclor', CECf, states, '0100')
CTX = Drug('Cefotaxime', CTXf, states, '1010')
ZOX = Drug('Ceftizoxime', ZOXf, states, '1001')
CXM = Drug('Cefuroxime', CXMf, states, '0100')
CRO = Drug('Ceftriaxone', CROf, states, '0100')
AMC = Drug('Amoxicillin+Clav', AMCf, states, '0100')
CAZ = Drug('Ceftazidime', CAZf, states, '0011')
CTT = Drug('Cefotetan', CTTf, states, '1101')
SAM = Drug('Ampicillin+Sulbactam', SAMf, states, '1111')
CPR = Drug('Cefprozil', CPRf, states, '0011')
CPD = Drug('Cefpodoxime', CPDf, states, '1010')
TZP = Drug('Piperacillin+Tazobactam', TZPf, states, '1000')
FEP = Drug('Cefepime', FEPf, states, '0000')

Now, we are going to find a metric 'Temporal Asymmetry' that shows the imbalance of drug applications to reach a minumum average fitness. 

Instead of using contour plots, as attempted previously, it is much more relevant to just use the matshow method.

This does not interpolate, as countour did, and each square represents the real discrete value.

### For example,
this is a 10x10 matrix of drug applications in the following order:

SAM $\rightarrow$ CPR (Steering)

AMP (Applied 10 times after steering sequence.)

The matrix shows the average fitness after the steering combination applied 1 to 10 times. Just for clarification, the order is SAM, CPR, AMP in all cases for this example.

In [ ]:
Ap = 10
avgMat = np.zeros([Ap+1, Ap+1])
for NA in range(0,Ap+1):
    for NB in range(0,Ap+1):
        SN = np.array(S0) * SAM.tMat**NA * CPR.tMat**NB * AMP.tMat**10
        AvgFit = np.dot(np.array(SN[0,:]), AMP.Fit)
        avgMat[NA, NB] = AvgFit

plt.set_cmap('YlOrBr')
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(111)
Z = ax.matshow(avgMat)
ax.set_xlabel('CPR Applied NB Times')
ax.set_ylabel('SAM Applied NA Times')
ax.set_title('Average Fitness <f>')
plt.colorbar(Z)

If the steering order were to be flopped (CPR$\rightarrow$SAM), the result would be very different.

In [ ]:
Ap = 10
avgMat = np.zeros([Ap+1, Ap+1])
for NA in range(0,Ap+1):
    for NB in range(0,Ap+1):
        SN = np.array(S0) * CPR.tMat**NA * SAM.tMat**NB * AMP.tMat**10
        AvgFit = np.dot(np.array(SN[0,:]), AMP.Fit)
        avgMat[NA, NB] = AvgFit
        
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(111)
Z = ax.matshow(avgMat)
ax.set_xlabel('SAM Applied NB Times')
ax.set_ylabel('CPR Applied NA Times')
ax.set_title('Average Fitness <f>')
plt.colorbar(Z)

In [ ]:
# list of all drug Classes
allDrugs = [AMP, AM, CEC, CTX, ZOX , CXM, CRO, AMC, CAZ, CTT, SAM, CPR, CPD, TZP, FEP]

In [ ]:
# Make a path for these plots, just for a test
if os.path.exists('Plots/') == False:
    os.makedirs('Plots/')

In [ ]:
# Change this to Y to plot all the matrices
Q = 'N'

These plots are all the possible combinations of steering drugs to the final drug shown in a 10x10 matrix. These are useful at the end when confirming asymmetry in application of drugs.

In [ ]:
if Q.upper() == 'N':
    for DRUG in allDrugs:
        # Drug we are steering to
        
        # Make a path for these plots, just for a test
        if os.path.exists('Plots/'+DRUG.name+'/') == False:
            os.makedirs('Plots/'+DRUG.name+'/')

        for i in range(len(allDrugs)):
            for j in range(len(allDrugs)):
                if allDrugs[i] != DRUG:
                    if allDrugs[j] != DRUG:
                        Ap = 10
                        avgMat = np.zeros([Ap+1, Ap+1])
                        for NA in range(0,Ap+1):
                            for NB in range(0,Ap+1):
                                SN = np.array(S0) * allDrugs[i].tMat**NA * allDrugs[j].tMat**NB * DRUG.tMat**10
                                AvgFit = np.dot(np.array(SN[0,:]), AMP.Fit)
                                avgMat[NA, NB] = AvgFit
                    
                        fig = plt.figure(figsize=(6,6))
                        ax = fig.add_subplot(111)
                        Z = ax.matshow(avgMat)
                        ax.set_xlabel(allDrugs[j].name + ' Applied Second N Times')
                        ax.set_ylabel(allDrugs[i].name + ' Applied First N Times')
                        ax.set_title('Average Fitness <f> for ' + DRUG.name)
                        plt.colorbar(Z)
                        plt.tight_layout()
                        plt.savefig('Plots/'+DRUG.name+'/'+allDrugs[i].name+'_'+allDrugs[j].name+'.png')
                        plt.close()

### Recording a metric
This next section attempts to find the location of the minimum peak fitness, and if that does not succeed, just the minimum average fitness. Then, the data is saved to an excel sheet for easy viewing.

Note: In future iterations, instead of selecting the lowest fitness, it might be beneficial to keep expanding the percent error.

In [ ]:
col_names=['Final Drug', 'Steering Drug 1', 'Steering Drug 1 Amount', 'Steering Drug 2', 'Steering Drug 2 Amount',
           'Drug1:Drug2', 'Drug1+Drug2', 'Lowest Fitness Peak Value', 'Lowest Fitness Achieved',
           'Maximum Simulated Drug Application', 'R-Value', 'Percent to LPFf Attempted']

In [ ]:
Ap = 10

book = xlwt.Workbook()
sh1 = book.add_sheet('1')
for n in range(len(col_names)):
    sh1.write(0, n, col_names[n])
    
K = 1
# Drug we are steering to
for DRUG in allDrugs:
    LPFf = DRUG.LPFf
    Perc = 0.1

    for i in range(len(allDrugs)):
        for j in range(len(allDrugs)):
            if allDrugs[i] != DRUG:
                if allDrugs[j] != DRUG:
                    avgMat = np.zeros([Ap+1, Ap+1])
                    for NA in range(0,Ap+1):
                        for NB in range(0,Ap+1):
                            SN = np.array(S0) * allDrugs[i].tMat**NA * allDrugs[j].tMat**NB * DRUG.tMat**10
                            AvgFit = np.dot(np.array(SN[0,:]), AMP.Fit)
                            avgMat[NA, NB] = AvgFit
                    Z = np.where(np.logical_and(avgMat<=LPFf*(1+Perc), avgMat>=LPFf*(1-Perc)))
                    indx = [[r, c] for r in Z[0] for c in Z[1]]
                    if len(indx) == 0:
                        Z = np.where(avgMat==avgMat.min())
                        indx = [[r, c] for r in Z[0] for c in Z[1]]
                        vals = [DRUG.name, allDrugs[i].name, indx[0][0], allDrugs[j].name, indx[0][1],
                                indx[0][0]/indx[0][1], indx[0][0]+indx[0][1], LPFf, avgMat.min(),
                                Ap, DRUG.R, Perc]
                    else:
                        vals = [DRUG.name, allDrugs[i].name, indx[0][0], allDrugs[j].name, indx[0][1],
                                indx[0][0]/indx[0][1], indx[0][0]+indx[0][1], LPFf,
                                avgMat[indx[0][0], indx[0][1]], Ap, DRUG.R, Perc]
#                     print(Z)  #  This was to ensure that the first index combo is always the lowest
#                     print(indx[0][0], indx[0][1])  #  And to ensure that this is the first index...
                    if np.isinf(vals[5]) == True:
                        vals[5] = -2
                    if np.isnan(vals[5]) == True:
                        vals[5] = -1
                    for n in range(len(vals)):
                        try:
                            sh1.write(K, n, vals[n])
                        except Exception:
                            vals[n] = vals[n].astype(np.float)
                            sh1.write(K, n, vals[n])
                    K = K + 1
                        
book.save('Plots/Temporal_AsymmetryR0.xls')

From the excel file, a dataframe is an easy way to visualize each series of drug applications.

In [ ]:
df = pd.read_excel('Plots/Temporal_AsymmetryR0.xls')
# Limit to show first five rows
df.head()

The following cell is used to plot the amounts of steering drug 1 and steering drug 2 needed to reach the tolerance of the lowest fitness peak. It shows that asymmetry exists in almost all cases, and that values for Amoxicillin and Cefotaxime should be double-checked.

In [ ]:
ggplot(aes(x='Steering Drug 1 Amount', y='Steering Drug 2 Amount'), data=df) +\
        facet_wrap('Final Drug') +\
        geom_point() +\
        scale_x_continuous(limits = (0, Ap)) +\
        scale_y_continuous(limits = (0, Ap)) +\
        ggtitle('R=0')

The default R-value is 0; however, the data would be much more interesting if it were to be set to 1. Also, the current applications is maxed at 10 to save computer memory. The above plots could show that we picked one of the best final steered drugs to attempt to analyze asymmetry. Or, we have to run the simulation longer.

## Changing R = 1; Application('Ap') = 10; Percent within = 10%

In [ ]:
r = 1

AMP = Drug('Ampicillin', AMPf, states, '0110', R=r)
AM = Drug('Amoxicillin', AMf, states, '0010', R=r)
CEC = Drug('Cefaclor', CECf, states, '0100', R=r)
CTX = Drug('Cefotaxime', CTXf, states, '1010', R=r)
ZOX = Drug('Ceftizoxime', ZOXf, states, '1001', R=r)
CXM = Drug('Cefuroxime', CXMf, states, '0100', R=r)
CRO = Drug('Ceftriaxone', CROf, states, '0100', R=r)
AMC = Drug('Amoxicillin+Clav', AMCf, states, '0100', R=r)
CAZ = Drug('Ceftazidime', CAZf, states, '0011', R=r)
CTT = Drug('Cefotetan', CTTf, states, '1101', R=r)
SAM = Drug('Ampicillin+Sulbactam', SAMf, states, '1111', R=r)
CPR = Drug('Cefprozil', CPRf, states, '0011', R=r)
CPD = Drug('Cefpodoxime', CPDf, states, '1010', R=r)
TZP = Drug('Piperacillin+Tazobactam', TZPf, states, '1000', R=r)
FEP = Drug('Cefepime', FEPf, states, '0000', R=r)

# list of all drug Classes
allDrugs = [AMP, AM, CEC, CTX, ZOX , CXM, CRO, AMC, CAZ, CTT, SAM, CPR, CPD, TZP, FEP]

book = xlwt.Workbook()
sh1 = book.add_sheet('1')
for n in range(len(col_names)):
    sh1.write(0, n, col_names[n])
    
K = 1
# Drug we are steering to
for DRUG in allDrugs:
    LPFf = DRUG.LPFf
    Perc = 0.1

    for i in range(len(allDrugs)):
        for j in range(len(allDrugs)):
            if allDrugs[i] != DRUG:
                if allDrugs[j] != DRUG:
                    Ap = 10
                    avgMat = np.zeros([Ap+1, Ap+1])
                    for NA in range(0,Ap+1):
                        for NB in range(0,Ap+1):
                            SN = np.array(S0) * allDrugs[i].tMat**NA * allDrugs[j].tMat**NB * DRUG.tMat**10
                            AvgFit = np.dot(np.array(SN[0,:]), AMP.Fit)
                            avgMat[NA, NB] = AvgFit
                    Z = np.where(np.logical_and(avgMat<=LPFf*(1+Perc), avgMat>=LPFf*(1-Perc)))
                    indx = [[r, c] for r in Z[0] for c in Z[1]]
                    if len(indx) == 0:
                        Z = np.where(avgMat==avgMat.min())
                        indx = [[r, c] for r in Z[0] for c in Z[1]]
                        vals = [DRUG.name, allDrugs[i].name, indx[0][0], allDrugs[j].name, indx[0][1],
                                indx[0][0]/indx[0][1], indx[0][0]+indx[0][1], LPFf, avgMat.min(),
                                Ap, DRUG.R, Perc]
                    else:
                        vals = [DRUG.name, allDrugs[i].name, indx[0][0], allDrugs[j].name, indx[0][1],
                                indx[0][0]/indx[0][1], indx[0][0]+indx[0][1], LPFf,
                                avgMat[indx[0][0], indx[0][1]], Ap, DRUG.R, Perc]
                    if np.isinf(vals[5]) == True:
                        vals[5] = -2
                    if np.isnan(vals[5]) == True:
                        vals[5] = -1
                    for n in range(len(vals)):
                        try:
                            sh1.write(K, n, vals[n])
                        except Exception:
                            vals[n] = vals[n].astype(np.float)
                            sh1.write(K, n, vals[n])
                    K = K + 1
                        
book.save('Plots/Temporal_AsymmetryR1.xls')

In [ ]:
df = pd.read_excel('Plots/Temporal_AsymmetryR1.xls')
df.head()

In [ ]:
ggplot(aes(x='Steering Drug 1 Amount', y='Steering Drug 2 Amount'), data=df) +\
        facet_wrap('Final Drug') +\
        geom_point() +\
        scale_x_continuous(limits = (0, Ap)) +\
        scale_y_continuous(limits = (0, Ap)) +\
        ggtitle('R=1')

This switching from R=0 to R=1 did not change much; however, that could be due to a high tolerance and low application.

### Closer look at Ampicillin
With some manipulation of the dataframe, a symmetric application of steering drugs is more abnormal than normal when attempting to reach lowest fitness peak.

In [ ]:
df = pd.read_excel('Plots/Temporal_AsymmetryR0.xls')
t = df[(df['Final Drug']=='Ampicillin')]
t = t[t['Drug1+Drug2'] != 0 ]

ggplot(aes(x='Steering Drug 1 Amount', y='Steering Drug 2 Amount',
           color='Steering Drug 1'), data=t) +\
        geom_point() +\
        scale_x_continuous(limits = (0, 11)) +\
        scale_y_continuous(limits = (0, 11)) +\
        ggtitle('Ampicillin')

The following sorting method separates out all the steering drugs and shows the highest sum first. 

To see the matrices of these plots see the Ampicillin directory:
- Ampicillin+Sulbactam_Cefprozil.png
- Amoxicillin+Clav_Cefotaxime.png
- Cefpodoxime_Cefuroxime.png
- Piperacillin+Tazobactam_Ceftizoxime.png
- Ceftriaxone_Cefuroxime.png

The Drug1:Drug2 is also important in assessing the magnitude of the asymmetry.

In [ ]:
rank = t[['Steering Drug 1', 'Steering Drug 1 Amount', 'Steering Drug 2', 'Steering Drug 2 Amount',
          'Drug1+Drug2', 'Drug1:Drug2', 'Lowest Fitness Achieved']]
rank = rank.sort_values(by='Drug1+Drug2', ascending=False)
print('Top Asymmetry by Addition for Ampicillin: ' + SAM.name + ' and ' + CPR.name)
print('Aiming for fitness value of {:.3f}'.format(AMP.LPFf))
rank.head(10)

## Changing R. One Drug and Steering Sequence at a time.
The following cells show how varying R from 0 to 50 affects the necessary drug applications.

In [ ]:
Ap = 10
K = 1
Perc = 0.0

book = xlwt.Workbook()
sh1 = book.add_sheet('1')
for n in range(len(col_names)):
    sh1.write(0, n, col_names[n])

for r in range(0, 11):

    AMP = Drug('Ampicillin', AMPf, states, '0110', R=r)
    AM = Drug('Amoxicillin', AMf, states, '0010', R=r)
    CEC = Drug('Cefaclor', CECf, states, '0100', R=r)
    CTX = Drug('Cefotaxime', CTXf, states, '1010', R=r)
    ZOX = Drug('Ceftizoxime', ZOXf, states, '1001', R=r)
    CXM = Drug('Cefuroxime', CXMf, states, '0100', R=r)
    CRO = Drug('Ceftriaxone', CROf, states, '0100', R=r)
    AMC = Drug('Amoxicillin+Clav', AMCf, states, '0100', R=r)
    CAZ = Drug('Ceftazidime', CAZf, states, '0011', R=r)
    CTT = Drug('Cefotetan', CTTf, states, '1101', R=r)
    SAM = Drug('Ampicillin+Sulbactam', SAMf, states, '1111', R=r)
    CPR = Drug('Cefprozil', CPRf, states, '0011', R=r)
    CPD = Drug('Cefpodoxime', CPDf, states, '1010', R=r)
    TZP = Drug('Piperacillin+Tazobactam', TZPf, states, '1000', R=r)
    FEP = Drug('Cefepime', FEPf, states, '0000', R=r)

    # list of all drug Classes
    allDrugs = [AMP, AM, CEC, CTX, ZOX , CXM, CRO, AMC, CAZ, CTT, SAM, CPR, CPD, TZP, FEP]
        
    # Drug we are steering to
    # for DRUG in allDrugs:
    DRUG = AMP
    LPFf = DRUG.LPFf

    for i in range(len(allDrugs)):
        for j in range(len(allDrugs)):
            if allDrugs[i] != DRUG:
                if allDrugs[j] != DRUG:
                    avgMat = np.zeros([Ap+1, Ap+1])
                    for NA in range(0,Ap+1):
                        for NB in range(0,Ap+1):
                            SN = np.array(S0) * allDrugs[i].tMat**NA * allDrugs[j].tMat**NB * DRUG.tMat**10
                            AvgFit = np.dot(np.array(SN[0,:]), AMP.Fit)
                            avgMat[NA, NB] = AvgFit
                    Z = np.where(np.logical_and(avgMat<=LPFf*(1+Perc), avgMat>=LPFf*(1-Perc)))
                    indx = [[r, c] for r in Z[0] for c in Z[1]]
                    if len(indx) == 0:
                        Z = np.where(avgMat==avgMat.min())
                        indx = [[r, c] for r in Z[0] for c in Z[1]]
                        vals = [DRUG.name, allDrugs[i].name, indx[0][0], allDrugs[j].name, indx[0][1],
                                indx[0][0]/indx[0][1], indx[0][0]+indx[0][1], LPFf, avgMat.min(),
                                Ap, DRUG.R, Perc]
                    else:
                        vals = [DRUG.name, allDrugs[i].name, indx[0][0], allDrugs[j].name, indx[0][1],
                                indx[0][0]/indx[0][1], indx[0][0]+indx[0][1], LPFf,
                                avgMat[indx[0][0], indx[0][1]], Ap, DRUG.R, Perc]
                    if np.isinf(vals[5]) == True:
                        vals[5] = -2
                    if np.isnan(vals[5]) == True:
                        vals[5] = -1
                    for n in range(len(vals)):
                        try:
                            sh1.write(K, n, vals[n])
                        except Exception:
                            vals[n] = vals[n].astype(np.float)
                            sh1.write(K, n, vals[n])
                    K = K + 1
                        
book.save('Plots/Temporal_Asymmetry_Amp_Rchange_0-50-1.xls')

In [ ]:
df = pd.read_excel('Plots/Temporal_Asymmetry_Amp_Rchange_0-50-1.xls')
df = df[df['Drug1+Drug2'] != 0]
stSAMCPR = df[(df['Steering Drug 1'] == SAM.name) & (df['Steering Drug 2'] == CPR.name)]
stSAMCPR.head(10)

The R-Values are plotted with different colors. The steering sequence is the familiar $SAM\rightarrow CPR\rightarrow AMP$.

In [ ]:
ggplot(aes(x='Steering Drug 1 Amount', y='Steering Drug 2 Amount',
           color='R-Value'), data=stSAMCPR) +\
        geom_point() +\
        scale_x_continuous(limits = (0, 11)) +\
        scale_y_continuous(limits = (0, 11)) +\
        ggtitle('Ampicillin Reaches Lowest Peak Fitness') +\
        xlab('SAM Applied') + ylab('CPR Applied')

In [ ]:
my_colors = ['#0000ff','#5922f2','#7a38e6',
             '#924bd9','#a65dcc','#b670be',
             '#c482b1','#d093a3','#daa594',
             '#e3b783','#ecc872','#f3da5d','#f9ec42','#ffff00']

# Only plot R=0 or R=1
dfR01 = df[df['R-Value'] == 0]
# dfR01
z = ggplot(dfR01, aes(x='Steering Drug 1 Amount', y='Steering Drug 2 Amount', color='Steering Drug 2')) +\
            geom_point(size=50) +\
            scale_color_manual(values=my_colors) +\
            facet_wrap('Steering Drug 1') +\
            scale_x_continuous(limits = (0, 11)) +\
            scale_y_continuous(limits = (0, 11)) +\
            ggtitle('Ampicillin Reaches Lowest Peak Fitness') +\
            xlab('Drug 1 Applied') + ylab('Drug 2 Applied')
z.save('pltR0.png', width=20, height=20, dpi=200)

In [ ]:
my_colors = ['#0000ff','#5922f2','#7a38e6',
             '#924bd9','#a65dcc','#b670be',
             '#c482b1','#d093a3','#daa594',
             '#e3b783','#ecc872','#f3da5d','#f9ec42','#ffff00']

# Only plot R=0 or R=1
dfR01 = df[df['R-Value'] == 1]
# dfR01
z = ggplot(dfR01, aes(x='Steering Drug 1 Amount', y='Steering Drug 2 Amount', color='Steering Drug 2')) +\
            geom_point(size=50) +\
            scale_color_manual(values=my_colors) +\
            facet_wrap('Steering Drug 1') +\
            scale_x_continuous(limits = (0, 11)) +\
            scale_y_continuous(limits = (0, 11)) +\
            ggtitle('Ampicillin Reaches Lowest Peak Fitness') +\
            xlab('Drug 1 Applied') + ylab('Drug 2 Applied')
z.save('pltR1.png', width=20, height=20, dpi=200)

## Next Steps...

$\rho$ decreasing with R and flattening out

$\phi$ dependence on R

Nice graph:

Polar where every combo is a $\rho$ vs $\phi$ scatter plot.

Each plot different for R = 0, 1, 2...

Can $\rho$ or $\phi$ be predicted from the structure of the drug matrix.

Keyword: eigenspectrum

$f_{ABC}(x, y, \infty) = \epsilon f_{ABC}(\infty, \infty, \infty)$